In [1]:
import pandas as pd
import seaborn as sns
import time
import re
import nltk
import math
import os
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def fasta_parser(filename):
    if os.path.exists(filename):
        pass
    else:                     
        print("The file, %s, does not exist" % filename)
        
    file = open(filename,mode='r')
    rec_all = file.read()
    file.close()
    # remove all whitespace from string all_of_it
    rec_all = rec_all.replace(' ','')
    # split records by > 
    records = rec_all.split('>')
    # Parse out the headers & sequences for each record
    headers = []
    sequences = []
    for rec in records:
        s = ''
        data = rec.split('\n')
        sq = s.join(data[1:])
        if len(data[0]) > 0:
            headers.append(data[0])
   
        if (len(sq) > 0):
            sequences.append(s.join(data[1:]))
    
    return headers, sequences

In [3]:
# parse extracellular protein fasta data
ex_head, ex_seqs = fasta_parser('extracell.fasta')

In [4]:
for head in ex_head:
    print(head)
    print()

P47710

P42127

O15444

P01042

Q9NRM1

P07360

P12110

P02786

P80303

P02788

P20061

P22749

Q13790

P26022

Q75N90

P35858

P11684

P11686

O43508

O95445

Q8IZN7

Q9HCQ7

Q9HBE4

P49908

P10451

Q9NQ38

P49767

Q96PH6

Q15113

Q9UBU3

P41271

Q76LX8

P49913

P06727

Q9UHD0

P04114

P24001

Q9NP70

P04118

P61812

P31151

P78556

P80188

Q10471

P01579

P25311

P43026

Q02383

P20809

P01210

P29459

Q15127

P01215

Q9BSG0

P01148

P06734

Q96A98

P29965

Q16363

Q9NZW4

O95399

Q03403

Q9NZH6

Q86VB7

P01583

P29460

P01584

P55268

P20742

Q9BX93

P01588

Q15063

P41439

P28300

Q8WX39

P54108

P27797

Q9UHF0

Q16378

P60022

Q9BXX0

Q9NPH9

P22352

Q9BQ08

P00797

P01303

P07988

Q8NET1

P01160

P01233

P16442

P01236

P10997

Q9NRR1

O15130

P04141

Q9GZT5

Q15726

P05019

P04217

P48061

Q9Y5X9

Q9NQ76

Q9Y240

Q8N729

P07998

Q9ULZ1

P48645

P01241

O00292

P22003

O43555

O94907

Q92743

P07492

P08294

P61278

Q99935

P04155

P07498

O94769

P19438

P40225

P10646

Q9Y251



In [5]:
for sq in ex_seqs:
    print(sq)
    print()

...



In [6]:
prot ={}
for idx in range(len(ex_head)):
    prot[ex_head[idx]] = ex_seqs[idx]

In [7]:
prot

{'O00292': 'MWPLWLCWALWVLPLAGPGAALTEEQLLGSLLRQLQLSEVPVLDRADMEKLVIPAHVRAQYVVLLRRSHGDRSRGKRFSQSFREVAGRFLASEASTHLLVFGMEQRLPPNSELVQAVLRLFQEPVPKAALHRHGRLSPRSAQARVTVEWLRVRDDGSNRTSLIDSRLVSVHESGWKAFDVTEAVNFWQQLSRPRQPLLLQVSVQREHLGPLASGAHKLVRFASQGAPAGLGEPQLELHTLDLRDYGAQGDCDPEAPMTEGTRCCRQEMYIDLQGMKWAKNWVLEPPGFLAYECVGTCQQPPEALAFNWPFLGPRQCIASETASLPMIVSIKEGGRTRPQVVSLPNMRVQKCSCASDGALVPRRLQP',
 'O00515': 'MAVSRKDWSALSSLARQRTLEDEEEQERERRRRHRNLSSTTDDEAPRLSQNGDRQASASERLPSVEEAEVPKPLPPASKDEDEDIQSILRTRQERRQRRQVVEAAQAPIQERLEAEEGRNSLSPVQATQKPLVSKKELEIPPRRRLSREQRGPWPLEEESLVGREPEERKKGVPEKSPVLEKSSMPKKTAPEKSLVSDKTSISEKVLASEKTSLSEKIAVSEKRNSSEKKSVLEKTSVSEKSLAPGMALGSGRRLVSEKASIFEKALASEKSPTADAKPAPKRATASEQPLAQEPPASGGSPATTKEQRGRALPGKNLPSLAKQGASDPPTVASRLPPVTLQVKIPSKEEEADMSSPTQRTYSSSLKRSSPRTISFRMKPKKENSETTLTRSASMKLPDNTVKLGEKLERYHTAIRRSESVKSRGLPCTELFVAPVGVASKRHLFEKELAGQSRAEPASSRKENLRLSGVVTSRLNLWISRTQESGDQDPQEAQKASSATERTQWGQKSDSSLDAEV',
 'O00626': 'MARLQTALLVVLVLLAVALQATEAGPYGANMEDSVCCRDYVRYRLPLRVVKHFYWTSDSCPRPGVVLLTFRDKEI

In [8]:
mito_head, mito_seqs = fasta_parser('mitochondrion.fasta')

In [9]:
for hd in mito_head:
    print(hd)
    print()

Q16762

Q9P0J0

O00165

P55809

Q9P0J6

P82912

Q16698

Q9NTG7

P82914

P49821

P54868

Q00325

P10515

O95298

Q9HAV7

Q96RP9

O00746

O95299

Q9NZE8

P30405

P11177

P17568

Q8N4E7

P61221

P16260

P82921

Q92552

O75489

Q9H9P8

P78540

P06576

Q9UI17

Q96EY1

Q8NI60

O60783

P51398

P07954

P51970

Q9BYC8

Q07021

Q9BYC9

Q9BXF6

Q16854

P49406

Q15118

P00403

P56556

P22830

P82930

Q14249

P82932

Q9UBF8

P82933

Q9HCC0

Q8N983

P54886

Q96RR1

P39210

P22695

P10606

P50747

Q05932

Q99757

Q9UQ90

Q9BYD1

Q9BYD2

Q9BZ23

Q9BYD3

P00846

Q8TCS8

Q969P6

O75570

Q9H2W6

Q16795

P21397

P10109

P43897

O60437

O75648

Q16798

Q9P0M9

P80404

P19404

Q9BRJ2

P51553

Q7KZN9

P12074

Q9NR28

P48047

Q9NYK5

Q9BXH1

O00411

P53597

P20674

O43464

Q9HD33

Q96KJ9

Q92581

Q9HD36

Q9NP92

P14406

Q8N573

Q9Y234

Q9BXI2

Q9Y2Q9

Q99417

Q15070

O43615

P00505

P24539

Q9GZT3

P48201

Q9UJA2

P00367

Q92665

Q9Y676

O75306

P31327

Q9Y2R5

Q9NX20

P09669

P50416

Q16891

P36776

Q6NZI2



In [10]:
for sq in mito_seqs:
    print(sq)
    print()

VHQVLYRALVSTKWLAESIRTGKLGPGLRVLDASWYSPGTREARKEYLERHVPGASFFDIEECRDTASPYEMMLPSEAGFAEYVGRLGISNHTHVVVYDGEHLGSFYAPRVWWMFRVFGHRTVSVLNGGFRNWLKEGHPVTSEPSRPEPAVFKATLDRSLLKTYEQVLENLESKRFQLVDSRSQGRFLGTEPEPDAVGLDSGHIRGAVNMPFMDFLTEDGFEKGPEELRALFQTKKVDLSQPLIATCRKGVTACHVALAAYLCGKPDVAVYDGSWSEWFRRAPPESRVSQGKSEKA

AASKVKQDMPPPGGYGPIDYKRNLPRRGLSGYSMLAIGIGTLIYGHWSIMKWNRERRRLQIEDFEARIALLPLLQAETDRRTLQMLRENLEEEAIIMKDVPDWKVGESVFHTTRWVPPLIGELYGLRTTEEALHASHGFMWYT

MSLFDLFRGFFGFPGPRSHRDPFFGGMTRDEDDDEEEEEEGGSWGRGNPRFHSPQHPPEEFGFGFSFSPGGGIRFHDNFGFDDLVRDFNSIFSDMGAWTLPSHPPELPGPESETPGERLREGQTLRDSMLKYPDSHQPRIFGGVLESDARSESPQPAPDWGSQRPFHRFDDVWPMDPHPRTREDNDLDSQVSQEGLGPVLQPQPKSYFKSISVTKITKPDGIVEERRTVVDSEGRTETTVTRHEADSSPRGDPESPRPPALDDAFSILDLFLGRWFRSR

MAALKLLSSGLRLCASARGSGATWYKGCVCSFSTSAHRHTKFYTDPVEAVKDIPDGATVLVGGFGLCGIPENLIDALLKTGVKGLTAVSNNAGVDNFGLGLLLRSKQIKRMVSSYVGENAEFERQYLSGELEVELTPQGTLAERIRAGGAGVPAFYTPTGYGTLVQEGGSPIKYNKDGSVAIASKPREVREFNGQHFILEEAITGDFALVKAWKADRAGNVIFRKSARNFNLPMCKAAETTVVEVEEIVDIGAFAPEDIHIPQIYVHRLIKGEKYE

...

